# Day 2

Today, we will start using nf-core pipelines to find differentially abundant genes in our dataset. 
We are using data from the following paper: https://www.nature.com/articles/s41593-023-01350-3#Sec10

1. Please take some time to read through the paper and understand their approach, hypotheses and goals.

### What was the objective of the study?

Analyze the results of oxycodone withdrawal on a transcriptomic level on chronic pain afflicted organisms, to find out about the possibility of artificial transcriptomic regulation as a factor to reduce opiod addition and other withdrawal side effects.

### What do the conditions mean?

oxy: Treated with oxycodone


sal: Treated with saline (control)

### What do the genotypes mean?

SNI: prolonged spared nerve injury surgery (chronic pain)

Sham: control

## Imagine you are the bioinformatician in the group who conducted this study. They hand you the raw files and ask you to analyze them.

### What would you do?
- QC
- RNA read mapping
- RNA isotype and abundance estimation -> count matrix
- Differential analysis
- Enrichment analysis (GO/KEGG)

### Which groups would you compare to each other? Please also mention which outcome you would expect to see from each comparison.
- SNI-Oxy vs SNI-Sal: Addictive effects for SNI-Oxy. Pain-induced immidiate and lasting pertubations for SNI-Saline. 
- Sham-Oxy vs. Sham-Sal: Addictive effects for SNI-Oxy. No significant effects for Sham-Sal. (Control?)
- SNI-Oxy vs. Sham-Oxy: Addictive side effects for both, but lingering pain effects after withdrawal for SNI.
- SNI-Sal vs. Sham-Sal: Immidiate and lasting pain effects for SNI-Oxy, no significant effects for Sham-Sal.  


# Data Analysis

Your group gave you a very suboptimal excel sheet (conditions_runs_oxy_project.xlsx) to get the information you need for each run they uploaded to the SRA.<br>
So, instead of directly diving into downloading the data and starting the analysis, you first need to sort the lazy table.<br>
Use Python and Pandas to get the table into a more sensible order.<br>
Then, perform some overview analysis and plot the results
1. How many samples do you have per condition? 8
2. How many samples do you have per genotype? 8
3. How often do you have each condition per genotype? 4

In [33]:
import pandas as pd

df = pd.read_excel("conditions_runs_oxy_project.xlsx", index_col="Run")
df = df.fillna(False)
df = df.replace("x", True)
df.sort_values(by=["condition: Sal", "Genotype: SNI"])
df

/tmp/ipykernel_6565/1699006584.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/tmp/ipykernel_6565/1699006584.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("x", True)


,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham
Run,,,,,,,
SRR23195505,?,True,False,True,False,True,False
SRR23195506,?,True,False,False,True,False,True
SRR23195507,?,True,False,True,False,False,True
SRR23195508,?,True,False,False,True,True,False
SRR23195509,?,True,False,False,True,True,False
SRR23195510,?,True,False,True,False,True,False
SRR23195511,?,True,False,False,True,False,True
SRR23195512,?,True,False,True,False,False,True
SRR23195513,?,True,False,True,False,True,False


In [34]:
# alt format: run name, condition, genotype cols
df["Condition"] = ["Sal" if x else "Oxy" for x in df["condition: Sal"]]
df["Genotype"] = ["Sham" if x else "SNI" for x in df["Genotype: Sham"]]
df

,Patient,RNA-seq,DNA-seq,condition: Sal,Condition: Oxy,Genotype: SNI,Genotype: Sham,Condition,Genotype
Run,,,,,,,,,
SRR23195505,?,True,False,True,False,True,False,Sal,SNI
SRR23195506,?,True,False,False,True,False,True,Oxy,Sham
SRR23195507,?,True,False,True,False,False,True,Sal,Sham
SRR23195508,?,True,False,False,True,True,False,Oxy,SNI
SRR23195509,?,True,False,False,True,True,False,Oxy,SNI
SRR23195510,?,True,False,True,False,True,False,Sal,SNI
SRR23195511,?,True,False,False,True,False,True,Oxy,Sham
SRR23195512,?,True,False,True,False,False,True,Sal,Sham
SRR23195513,?,True,False,True,False,True,False,Sal,SNI


In [36]:
df_new = df[["Condition", "Genotype"]]
df_new = df_new.sort_values(by=["Condition", "Genotype"])
df_new

,Condition,Genotype
Run,,
SRR23195508,Oxy,SNI
SRR23195509,Oxy,SNI
SRR23195516,Oxy,SNI
SRR23195517,Oxy,SNI
SRR23195506,Oxy,Sham
SRR23195511,Oxy,Sham
SRR23195514,Oxy,Sham
SRR23195519,Oxy,Sham
SRR23195505,Sal,SNI


In [29]:
print("Oxy condition count: ", len(df_new[df_new["Condition"] == "Oxy"]))
print("SNI genotype count: ", len(df_new[df_new["Genotype"] == "SNI"]))

Oxy condition count:  8
SNI genotype count:  8


They were so kind to also provide you with the information of the number of bases per run, so that you can know how much space the data will take on your Cluster.<br>
Add a new column to your fancy table with this information (base_counts.csv) and sort your dataframe according to this information and the condition.

Then select the 2 smallest runs from your dataset and download them from SRA (maybe an nf-core pipeline can help here?...)

In [ ]:
bases = pd.read_csv("base_counts.csv", index_col="Run")
bases

,Bases
Run,
SRR23195505,6922564500
SRR23195506,7859530800
SRR23195507,8063298900
SRR23195508,6927786900
SRR23195509,7003550100
SRR23195510,7377388500
SRR23195511,6456390900
SRR23195512,7462857900
SRR23195513,8099181600


In [42]:
combined = df_new.merge(bases, on="Run").sort_values(["Bases"])
combined

,Condition,Genotype,Bases
Run,,,
SRR23195516,Oxy,SNI,6203117700
SRR23195511,Oxy,Sham,6456390900
SRR23195517,Oxy,SNI,6863840400
SRR23195505,Sal,SNI,6922564500
SRR23195508,Oxy,SNI,6927786900
SRR23195519,Oxy,Sham,6996050100
SRR23195509,Oxy,SNI,7003550100
SRR23195514,Oxy,Sham,7226808600
SRR23195510,Sal,SNI,7377388500


In [ ]:
smallest_sras = combined.iloc[0:2]
smallest_sras

,Condition,Genotype,Bases
Run,,,
SRR23195516,Oxy,SNI,6203117700
SRR23195511,Oxy,Sham,6456390900


In [59]:
!nextflow run nf-core/fetchngs --max_memory "4GB" -profile docker --input ids.csv --outdir fastq/ -resume


 N E X T F L O W   ~  version 25.04.7

Launching `https://github.com/nf-core/fetchngs` [insane_baekeland] DSL2 - revision: 8ec2d934f9 [master]

WARN: Access to undefined parameter `monochromeLogs` -- Initialise it to a default value eg. `params.monochromeLogs = some_value`


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/fetchngs v1.12.0-g8ec2d93
------------------------------------------------------
Core Nextflow options
  revision       : master
  runName        : insane_baekeland
  containerEngine: docker
  launchDir      : /home/marcelullose/Documents/SoSe25/ComputationalWorkflows/computational-workflows-2025/notebooks/day_02
  workDir        : /home/marcelullose/Documents/SoSe25/ComputationalWorkflows/computational-workfl

While your files are downloading, get back to the paper and explain how you would try to reproduce the analysis.<br>
When you are done with this shout, so we can discuss the different ideas.